In [ ]:
import pandas as pd
import numpy as np
import json
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
import re

os.chdir('/content/drive/My Drive/OEM/Final datasets')

In [ ]:
with open('covid_adverse.json') as f:
  scenario2_input_data = pd.DataFrame(json.loads(f.read()))
print(scenario2_input_data.shape)
scenario2_input_data.head()

(207484, 57)


,index,mdr_text_mdr_text_key,mdr_text_text_type_code,mdr_text_patient_sequence_number,mdr_text_text,device_manufacturer_d_address_1,device_manufacturer_d_address_2,device_device_event_key,device_device_sequence_number,device_manufacturer_d_state,device_manufacturer_d_zip_code,device_manufacturer_d_city,device_lot_number,device_manufacturer_d_postal_code,device_manufacturer_d_zip_code_ext,device_model_number,device_date_received,device_device_report_product_code,device_device_operator,device_device_availability,device_other_id_number,device_generic_name,device_manufacturer_d_name,device_manufacturer_d_country,device_brand_name,device_device_age_text,device_device_evaluated_by_manufacturer,device_catalog_number,device_implant_flag,device_date_removed_flag,device_date_returned_to_manufacturer,device_baseline_510_k__flag,device_baseline_510_k__number,device_baseline_510_k__exempt_flag,device_expiration_date_of_device,event_type,report_number,type_of_report,product_problem_flag,date_received,reprocessed_and_reused_flag,previous_use_code,mdr_report_key,report_source_code,device_date_of_manufacturer,single_use_flag,remedial_action,distributor_name,product_problems,manufacturer_link_flag,report_to_manufacturer,device_openfda.device_name,device_openfda.medical_specialty_description,device_openfda.device_class,device_openfda.regulation_number,device_openfda.registration_number,device_openfda.fei_number
0,0,189010823,Description of Event or Problem,1,WHILE IN STANDBY MODE THE VENTILATOR TURNED ON...,2271 COSMOS CT,,,1.0,CA,92011,CARLSBAD,,92011,,V60,20200427,MNT,HEALTH PROFESSIONAL,Yes,,"VENTILATOR, CONTINUOUS, MINIMAL VENTILATORY SU...","RESPIRONICS CALIFORNIA, LLC",US,PHILIPS RESPIRONICS,8 YR,*,,,,None,None,None,None,None,Malfunction,10000491,[Initial submission],Y,20200427,N,N,10000491,User Facility report,None,*,[],,None,N,,"Ventilator, Continuous, Minimal Ventilatory Su...",Anesthesiology,2,868.5895,None,None
1,1,189011437,Description of Event or Problem,1,"DURING STAGING OF THE BIPAP VENTILATOR, THE VE...",2271 COSMOS CT,,,1.0,CA,92011,CARLSBAD,,92011,,V60,20200427,MNT,HEALTH PROFESSIONAL,Yes,,"VENTILATOR, CONTINUOUS, MINIMAL VENTILATORY SU...","RESPIRONICS CALIFORNIA, LLC",US,PHILIPS RESPIRONICS,9 YR,*,,,,None,None,None,None,None,Malfunction,10000497,[Initial submission],Y,20200427,N,N,10000497,User Facility report,None,*,[],,None,N,,"Ventilator, Continuous, Minimal Ventilatory Su...",Anesthesiology,2,868.5895,None,None
2,2,189072753,Description of Event or Problem,1,A VENTILATOR WAS RETURNED TO THE MANUFACTURER ...,1001 MURRY RIDGE LANE,,,1.0,,,"MURRYSVILLE, PA",,,,1040000,20200427,CBK,HEALTH PROFESSIONAL,Device was returned to manufacturer,,"VENTILATOR, CONTINUOUS, FACILITY USE",RESPIRONICS INC.,US,TRILOGY 202,DA,Y,1040000,,,20200416,None,None,None,None,Malfunction,2518422-2020-01076,"[Initial submission, Followup]",Y,20200427,N,U,10000611,Manufacturer report,20161221,N,[],,[Circuit Failure],Y,,"Ventilator, Continuous, Facility Use",Anesthesiology,2,868.5895,[2518422],[2518422]
3,3,191796414,Additional Manufacturer Narrative,1,"IT WAS INITIALLY REPORTED, DURING THE EVALUATI...",1001 MURRY RIDGE LANE,,,1.0,,,"MURRYSVILLE, PA",,,,1040000,20200427,CBK,HEALTH PROFESSIONAL,Device was returned to manufacturer,,"VENTILATOR, CONTINUOUS, FACILITY USE",RESPIRONICS INC.,US,TRILOGY 202,DA,Y,1040000,,,20200416,None,None,None,None,Malfunction,2518422-2020-01076,"[Initial submission, Followup]",Y,20200427,N,U,10000611,Manufacturer report,20161221,N,[],,[Circuit Failure],Y,,"Ventilator, Continuous, Facility Use",Anesthesiology,2,868.5895,[2518422],[2518422]
4,4,189073684,Description of Event or Problem,1,THE MANUFACTURER RECEIVED INFORMATION ALLEGING...,1001 MURRY RIDGE LANE,,,1.0,,,"MURRYSVILLE, PA",,,,1054096,20200427,CBK,HEALTH PROFESSIONAL,Device was returned to manufacturer,,"VENTILATOR, CONTINUOUS, FACILITY USE",RESPIRONICS INC,US,TRILOGY 100,,Y,1054096,,,20200415,None,None,None,None,Malfunction,2518422-2020-01065,[Initial submission],Y,2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
scenario2_input_data.isna().sum()

index                                                0
mdr_text_mdr_text_key                                0
mdr_text_text_type_code                              0
mdr_text_patient_sequence_number                     0
mdr_text_text                                        0
device_manufacturer_d_address_1                      0
device_manufacturer_d_address_2                      0
device_device_event_key                              0
device_device_sequence_number                        0
device_manufacturer_d_state                          0
device_manufacturer_d_zip_code                       0
device_manufacturer_d_city                           0
device_lot_number                                    0
device_manufacturer_d_postal_code                    0
device_manufacturer_d_zip_code_ext                   0
device_model_number                                  0
device_date_received                                 0
device_device_report_product_code                    0
device_dev

In [ ]:
scenario2_input_data.columns

Index(['mdr_text_text_type_code', 'mdr_text_patient_sequence_number',
       'device_manufacturer_d_address_1', 'device_device_sequence_number',
       'device_manufacturer_d_state', 'device_manufacturer_d_city',
       'device_manufacturer_d_zip_code_ext', 'device_model_number',
       'device_date_received', 'device_device_report_product_code',
       'device_device_operator', 'device_other_id_number',
       'device_generic_name', 'device_manufacturer_d_name',
       'device_manufacturer_d_country', 'device_brand_name',
       'device_date_removed_flag', 'event_type', 'product_problem_flag',
       'date_received', 'reprocessed_and_reused_flag', 'previous_use_code',
       'device_date_of_manufacturer', 'single_use_flag', 'remedial_action',
       'device_openfda.device_name',
       'device_openfda.medical_specialty_description',
       'device_openfda.device_class', 'device_openfda.regulation_number',
       'device_openfda.registration_number', 'device_openfda.fei_number'],
     

In [ ]:
scenario2_input_data = scenario2_input_data.drop(['index', 'mdr_text_mdr_text_key', 'mdr_text_text', 'device_lot_number', 'device_implant_flag', 'device_baseline_510_k__flag', 'device_baseline_510_k__number', 'device_baseline_510_k__exempt_flag', 'device_manufacturer_d_postal_code', 'device_device_availability', 'device_device_age_text', 'device_catalog_number', 'device_date_returned_to_manufacturer', 'device_expiration_date_of_device', 'report_number', 'type_of_report', 'mdr_report_key', 'report_source_code', 'distributor_name',
    'product_problems', 'mdr_text_text_type_code', 'mdr_text_patient_sequence_number', 'device_device_sequence_number', 'device_manufacturer_d_address_1', 'device_date_received','date_received', 'device_manufacturer_d_zip_code', 'device_device_event_key','device_date_of_manufacturer', 'device_openfda.registration_number', 'device_openfda.regulation_number', 'device_openfda.fei_number', 'device_manufacturer_d_address_2', 'device_device_evaluated_by_manufacturer', 'manufacturer_link_flag', 'report_to_manufacturer', 'date_report_to_manufacturer'], axis=1, errors='ignore')

In [ ]:
scenario2_input_data.head()

,mdr_text_text_type_code,mdr_text_patient_sequence_number,device_manufacturer_d_address_1,device_device_sequence_number,device_manufacturer_d_state,device_manufacturer_d_city,device_manufacturer_d_zip_code_ext,device_model_number,device_date_received,device_device_report_product_code,device_device_operator,device_other_id_number,device_generic_name,device_manufacturer_d_name,device_manufacturer_d_country,device_brand_name,device_date_removed_flag,event_type,product_problem_flag,date_received,reprocessed_and_reused_flag,previous_use_code,device_date_of_manufacturer,single_use_flag,remedial_action,device_openfda.device_name,device_openfda.medical_specialty_description,device_openfda.device_class,device_openfda.regulation_number,device_openfda.registration_number,device_openfda.fei_number
0,Description of Event or Problem,1,2271 COSMOS CT,1.0,CA,CARLSBAD,,V60,20200427,MNT,HEALTH PROFESSIONAL,,"VENTILATOR, CONTINUOUS, MINIMAL VENTILATORY SU...","RESPIRONICS CALIFORNIA, LLC",US,PHILIPS RESPIRONICS,,Malfunction,Y,20200427,N,N,None,*,[],"Ventilator, Continuous, Minimal Ventilatory Su...",Anesthesiology,2,868.5895,None,None
1,Description of Event or Problem,1,2271 COSMOS CT,1.0,CA,CARLSBAD,,V60,20200427,MNT,HEALTH PROFESSIONAL,,"VENTILATOR, CONTINUOUS, MINIMAL VENTILATORY SU...","RESPIRONICS CALIFORNIA, LLC",US,PHILIPS RESPIRONICS,,Malfunction,Y,20200427,N,N,None,*,[],"Ventilator, Continuous, Minimal Ventilatory Su...",Anesthesiology,2,868.5895,None,None
2,Description of Event or Problem,1,1001 MURRY RIDGE LANE,1.0,,"MURRYSVILLE, PA",,1040000,20200427,CBK,HEALTH PROFESSIONAL,,"VENTILATOR, CONTINUOUS, FACILITY USE",RESPIRONICS INC.,US,TRILOGY 202,,Malfunction,Y,20200427,N,U,20161221,N,[],"Ventilator, Continuous, Facility Use",Anesthesiology,2,868.5895,[2518422],[2518422]
3,Additional Manufacturer Narrative,1,1001 MURRY RIDGE LANE,1.0,,"MURRYSVILLE, PA",,1040000,20200427,CBK,HEALTH PROFESSIONAL,,"VENTILATOR, CONTINUOUS, FACILITY USE",RESPIRONICS INC.,US,TRILOGY 202,,Malfunction,Y,20200427,N,U,20161221,N,[],"Ventilator, Continuous, Facility Use",Anesthesiology,2,868.5895,[2518422],[2518422]
4,Description of Event or Problem,1,1001 MURRY RIDGE LANE,1.0,,"MURRYSVILLE, PA",,1054096,20200427,CBK,HEALTH PROFESSIONAL,,"VENTILATOR, CONTINUOUS, FACILITY USE",RESPIRONICS INC,US,TRILOGY 100,,Malfunction,Y,20200427,N,U,20180213,N,[],"Ventilator, Continuous, Facility Use",Anesthesiology,2,868.5895,[2518422],[2518422]


In [ ]:
scenario2_input_data['op_flag'] = scenario2_input_data['device_date_returned_to_manufacturer'].notna().astype(int)

In [ ]:
scenario2_input_data = scenario2_input_data[['device_device_report_product_code', 'device_brand_name', 'distributor_name', 'device_manufacturer_d_name', 'device_catalog_number', 'device_openfda.medical_specialty_description', 'device_model_number', 'device_generic_name', 'op_flag']]

In [ ]:
scenario2_input_data.isna().sum()

device_device_report_product_code               0
device_brand_name                               0
distributor_name                                0
device_manufacturer_d_name                      0
device_catalog_number                           0
device_openfda.medical_specialty_description    7
device_model_number                             0
device_generic_name                             0
op_flag                                         0
dtype: int64

In [ ]:
scenario2_input_data = scenario2_input_data[scenario2_input_data['device.manufacturer_d_country'].notnull()]
scenario2_input_data.isna().sum()

mdr_text.text_type_code                           0
device.date_received                              0
device.device_report_product_code                 0
device.manufacturer_d_name                        1
device.manufacturer_d_country                     0
event_type                                        0
reprocessed_and_reused_flag                      17
previous_use_code                                94
single_use_flag                                 101
device.openfda.medical_specialty_description     11
op_flag                                           0
dtype: int64

In [ ]:
scenario2_input_data['single_use_flag'].value_counts()

N    9991
Y    4995
*     297
I      14
Name: single_use_flag, dtype: int64

In [ ]:
scenario2_input_data['single_use_flag'] = scenario2_input_data['single_use_flag'].fillna('N')

In [ ]:
scenario2_input_data['single_use_flag'].value_counts()

N    10092
Y     4995
*      297
I       14
Name: single_use_flag, dtype: int64

In [ ]:
scenario2_input_data['single_use_flag'] = scenario2_input_data['single_use_flag'].apply(lambda x : re.sub('I', 'Y', x))

In [ ]:
scenario2_input_data['single_use_flag'] = scenario2_input_data['single_use_flag'].astype(str)

In [ ]:
scenario2_input_data = scenario2_input_data[scenario2_input_data['single_use_flag'] != '*']

In [ ]:
scenario2_input_data.shape

(15101, 11)

In [ ]:
scenario2_input_data.isna().sum()

mdr_text.text_type_code                          0
device.date_received                             0
device.device_report_product_code                0
device.manufacturer_d_name                       1
device.manufacturer_d_country                    0
event_type                                       0
reprocessed_and_reused_flag                     17
previous_use_code                               94
single_use_flag                                  0
device.openfda.medical_specialty_description    11
op_flag                                          0
dtype: int64

In [ ]:
scenario2_input_data['previous_use_code'].value_counts()

I    7012
R    4784
U    2584
N     556
*      71
Name: previous_use_code, dtype: int64

In [ ]:
scenario2_input_data['previous_use_code'] = scenario2_input_data['previous_use_code'].fillna('N')

In [ ]:
scenario2_input_data['previous_use_code'].value_counts()

I    7012
R    4784
U    2584
N     650
*      71
Name: previous_use_code, dtype: int64

In [ ]:
scenario2_input_data = scenario2_input_data[scenario2_input_data['previous_use_code'] != '*']

In [ ]:
scenario2_input_data['previous_use_code'].value_counts()

I    7012
R    4784
U    2584
N     650
Name: previous_use_code, dtype: int64

In [ ]:
scenario2_input_data.isna().sum()

mdr_text.text_type_code                          0
device.date_received                             0
device.device_report_product_code                0
device.manufacturer_d_name                       1
device.manufacturer_d_country                    0
event_type                                       0
reprocessed_and_reused_flag                     17
previous_use_code                                0
single_use_flag                                  0
device.openfda.medical_specialty_description    11
op_flag                                          0
dtype: int64

In [ ]:
scenario2_input_data['reprocessed_and_reused_flag'].value_counts()

N    14379
I      619
Y       15
Name: reprocessed_and_reused_flag, dtype: int64

In [ ]:
scenario2_input_data['reprocessed_and_reused_flag'] = scenario2_input_data['reprocessed_and_reused_flag'].fillna('I')

In [ ]:
scenario2_input_data['reprocessed_and_reused_flag'].value_counts()

N    14379
I      636
Y       15
Name: reprocessed_and_reused_flag, dtype: int64

In [ ]:
scenario2_input_data.isna().sum()

mdr_text.text_type_code                          0
device.date_received                             0
device.device_report_product_code                0
device.manufacturer_d_name                       1
device.manufacturer_d_country                    0
event_type                                       0
reprocessed_and_reused_flag                      0
previous_use_code                                0
single_use_flag                                  0
device.openfda.medical_specialty_description    11
op_flag                                          0
dtype: int64

In [ ]:
scenario2_input_data = scenario2_input_data[scenario2_input_data['device.openfda.medical_specialty_description'].notnull()]

In [ ]:
scenario2_input_data = scenario2_input_data[scenario2_input_data['device.manufacturer_d_name'].notnull()]

In [ ]:
scenario2_input_data.isna().sum()

mdr_text.text_type_code                         0
device.date_received                            0
device.device_report_product_code               0
device.manufacturer_d_name                      0
device.manufacturer_d_country                   0
event_type                                      0
reprocessed_and_reused_flag                     0
previous_use_code                               0
single_use_flag                                 0
device.openfda.medical_specialty_description    0
op_flag                                         0
dtype: int64

In [ ]:
scenario2_input_data['device_manufacturer_d_name'].value_counts()

RESPIRONICS CALIFORNIA, INC    53374
COVIDIEN                       25839
RESPIRONICS, INC               13574
RESPIRONICS, INC.              13024
CAREFUSION, INC                10427
                               ...  
MONAGHAN MEDICAL CORP              1
MILLINCKRODT INC.                  1
MELLINCKRODT                       1
MEDTRONIC INC./ COVIDIEN           1
NELCOR PURITAN BENNETT             1
Name: device_manufacturer_d_name, Length: 1161, dtype: int64

In [ ]:
# scenario2_input_data['device_date_received'] =  pd.to_datetime(scenario2_input_data['device_date_received'])
for col in scenario2_input_data.columns:
    if col != 'device.date_received':
        scenario2_input_data[col] = scenario2_input_data[col].astype('category')

In [ ]:
scenario2_input_data.dtypes

device_device_report_product_code               category
device_brand_name                               category
distributor_name                                category
device_manufacturer_d_name                      category
device_catalog_number                           category
device_openfda.medical_specialty_description    category
device_model_number                             category
device_generic_name                             category
op_flag                                         category
dtype: object

### Encoding the data

In [ ]:
def dataEncoding(df):
    manuf_name_data = pd.get_dummies(df['device.manufacturer_d_name'])
    manuf_country_data = pd.get_dummies(df['device.manufacturer_d_country'])
    med_spl_data = pd.get_dummies(df['device.openfda.medical_specialty_description'])
    event_type_data = pd.get_dummies(df['event_type'])
    txt_type_data = pd.get_dummies(df['mdr_text.text_type_code'])
    reused_data = pd.get_dummies(df['reprocessed_and_reused_flag'])
    prev_used_data = pd.get_dummies(df['previous_use_code'])
    single_use_data = pd.get_dummies(df['single_use_flag'])
    df = pd.concat([df.drop(['mdr_text.text_type_code', 'device.date_received', 'device.manufacturer_d_name', 'device.manufacturer_d_country',
       'event_type', 'reprocessed_and_reused_flag', 'previous_use_code',
       'single_use_flag', 'device.openfda.medical_specialty_description'], axis=1, errors='ignore'), manuf_name_data, manuf_country_data, reused_data, event_type_data, prev_used_data, single_use_data, med_spl_data, txt_type_data], axis=1)
    return df

In [ ]:
manuf_name_data = pd.get_dummies(scenario2_input_data['device.manufacturer_d_name'])
manuf_name_data

,3M HEALTH CARE,ABBOTT,ABBOTT DIABETES CARE INC,"ABBOTT DIABETES CARE INC, USA",ABBOTT DIABETES CARE LIMITED UK,ABBOTT DIAGNOSTICS DIV/ CELL DYN,ABBOTT DIAGNOSTICS DIVISION,ABBOTT DIAGNOSTICS DIVISION/CELLDYN,ABBOTT DIAGNOSTICS DIVISON/CELLDYN,ABBOTT GMBH,ABBOTT MANUFACTURING INC,"ABBOTT MANUFACTURING, INC.","ABBOTT MOLECULAR, INC.",ABBOTT SPINE,ABBOTT VASCULAR,ABBOTT VASCULAR SWITZERLAND,ABBOTT VASCULAR-CARDIAC THERAPIES,ABBOTT VASCULAR-VASCULAR SOLUTIONS,"ABIOMED, INC.","ADVANCED BIONICS, LLC",ADVANCED STERILIZATION PRODUCTS,AESCULAP INC,AGA MEDICAL CORPORATION,AIR LIQUIDE HEALTHCARE AMERICA CORP.,"ALARIS MEDICAL SYSTEMS, INC.",ALCON - IRVINE TECH CTR,ALCON - IRVINE TECHNOLOGY CENTER,ALCON - ORLANDO TECHNOLOGY CENTER,"ALCON RESEARCH, LLC - HOUSTON","ALCON RESEARCH, LLC - HUNTINGTON","ALCON RESEARCH, LTD. / HUNTINGTON","ALCON RESEARCH, LTD/ HUNTINGTON","ALERE SAN DIEGO, INC.",ALLERGAN (IRVINE),ANGIODYNAMICS,ANGIOMED GMBH & CO. MEDIZINTECHNIK KG,ANIMAS CORP.,ANIMAS LLC,APPLIED MEDICAL RESOURCES,APPLIED MEDICAL RESOURCES CORP.,...,TERUMO CARDIOVASCULAR SYSTEMS CORP,TERUMO CARDIOVASCULAR SYSTEMS CORP.,"TERUMO CORPORATION, ASHITAKA",TERUMO MEDICAL CORP.,TERUMO MEDICAL CORPORATION,THORATEC CORP.,THORATEC CORPORATION,THORATEC SWITZERLAND GMBH,TOGO MEDIKIT CO. LTD.,TOSOH CORPORATION,TRAVERSE BAY MANUFACTURING,TRILLIANT SURGICAL,TRUMPF MEDIZIN SYSTEME GMBH + CO. KG,"ULTI-MED, INC.",US SURGICAL PUERTO RICO,VERATHON MEDICAL ULC,VYAIRE MEDICAL,VYGON USA,WARSAW ORTHOPEDIC INC.,WARSAW ORTHOPEDICS,"WELCH ALLYN PROTOCOL, INC.","WILLIAM A. COOK AUSTRALIA, PTY LTD",WILLIAM COOK EUROPE,"WRIGHT MEDICAL TECHNOLOGY, INC.",WRIGHTS LANE SYNTHES USA PRODUCTS LLC,ZIMMER BIOMET SPINE INC.,"ZIMMER BIOMET, INC.",ZIMMER DENTAL,ZIMMER GMBH,ZIMMER MANUFACTURING B.V.,ZIMMER ORTHOPAEDIC MFG. LTD.,ZIMMER SPINE,"ZIMMER SURGICAL, INC.",ZIMMER TMT,"ZIMMER, INC.",ZOLL CIRCULATION,ZOLL MANUFACTURING CORPORATION,ZOLL MEDICAL CORP,ZOLL MEDICAL CORP.,ZOLL MEDICAL CORPORATION
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17423,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17427,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17428,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17429,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
manuf_country_data = pd.get_dummies(scenario2_input_data['device.manufacturer_d_country'])
manuf_country_data

,*,AS,AU,BR,CA,CH,CS,DA,DE,DR,EI,FR,GM,HO,IE,IS,IT,JA,KR,MX,MY,NO,NZ,PL,PR,SE,SG,SN,SP,SW,SZ,TH,UK,US
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17423,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
17427,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
17428,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
17429,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
scenario2_input_data.drop(['device.device_report_product_code'], axis=1, inplace=True)

In [ ]:
med_spl_data = pd.get_dummies(scenario2_input_data['device.openfda.medical_specialty_description'])
med_spl_data

,Anesthesiology,Cardiovascular,Clinical Chemistry,Dental,"Ear, Nose, Throat","Gastroenterology, Urology",General Hospital,"General, Plastic Surgery",Hematology,Immunology,Microbiology,Neurology,Obstetrics/Gynecology,Ophthalmic,Orthopedic,Physical Medicine,Radiology,Unknown
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17423,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17427,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
17428,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
17429,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
scenario2_input_data['event_type'].value_counts()

Malfunction    13286
Injury          1702
Death             21
Other              9
Name: event_type, dtype: int64

In [ ]:
event_type_data = pd.get_dummies(scenario2_input_data['event_type'])
event_type_data

,Death,Injury,Malfunction,Other
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
17423,0,0,1,0
17427,0,0,1,0
17428,0,0,1,0
17429,0,0,1,0


In [ ]:
txt_type_data = pd.get_dummies(scenario2_input_data['mdr_text.text_type_code'])
txt_type_data

,Additional Manufacturer Narrative,Description of Event or Problem
0,1,0
1,1,0
2,0,1
3,1,0
4,0,1
...,...,...
17423,0,1
17427,1,0
17428,0,1
17429,1,0


In [ ]:
reused_data = pd.get_dummies(scenario2_input_data['reprocessed_and_reused_flag'])
reused_data

,I,N,Y
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
17423,0,1,0
17427,1,0,0
17428,1,0,0
17429,0,1,0


In [ ]:
prev_used_data = pd.get_dummies(scenario2_input_data['previous_use_code'])
prev_used_data

,I,N,R,U
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
17423,0,1,0,0
17427,0,0,0,1
17428,0,0,0,1
17429,0,0,0,1


In [ ]:
single_use_data = pd.get_dummies(scenario2_input_data['single_use_flag'])
single_use_data

,N,Y
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
17423,1,0
17427,1,0
17428,1,0
17429,1,0


In [ ]:
scenario2_input_data.columns

Index(['mdr_text.text_type_code', 'device.date_received',
       'device.manufacturer_d_name', 'device.manufacturer_d_country',
       'event_type', 'reprocessed_and_reused_flag', 'previous_use_code',
       'single_use_flag', 'device.openfda.medical_specialty_description',
       'op_flag'],
      dtype='object')

In [ ]:
scenario2_input_data = pd.concat([scenario2_input_data.drop(['mdr_text.text_type_code', 'device.manufacturer_d_name', 'device.manufacturer_d_country',
       'event_type', 'reprocessed_and_reused_flag', 'previous_use_code',
       'single_use_flag', 'device.openfda.medical_specialty_description'], axis=1), manuf_name_data, manuf_country_data, reused_data, event_type_data, prev_used_data, single_use_data, med_spl_data, txt_type_data], axis=1)

In [ ]:
scenario2_input_data.drop(['device.date_received'], axis=1, inplace=True)

In [ ]:
X = scenario2_input_data.iloc[:, :-1].values
y = scenario2_input_data.iloc[:, -1].values

In [ ]:
y

array([0, 0, 1, ..., 1, 0, 1], dtype=uint8)

### Model Creation

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=95)

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(objective = 'binary:logistic', learning_rate=0.2, n_estimators = 1000, max_delta = None)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta=None, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
from sklearn.metrics import accuracy_score

y_predict = xgb_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test, y_predict)
xgb_accuracy

1.0

In [ ]:
test_data = pd.read_csv('model2_test.csv')
test_data.isna().sum()

Unnamed: 0                                         0
mdr_text.mdr_text_key                              0
mdr_text.text_type_code                            0
mdr_text.text                                      0
device.manufacturer_d_state                      324
device.lot_number                                810
device.manufacturer_d_postal_code                273
device.model_number                              336
device.date_received                               0
device.device_report_product_code                  0
device.device_availability                         0
device.generic_name                               21
device.manufacturer_d_name                         4
device.manufacturer_d_country                     93
device.brand_name                                 15
device.date_returned_to_manufacturer            1306
device.device_age_text                          1145
device.catalog_number                            189
device.expiration_date_of_device              